# Automatizando la Construction de Decks the Magic - Commander

##### Maximize the number of combos subject to a budget constraint, and the selection of 100 cards

In [1]:
import urllib, json
import pandas as pd
import numpy as np
import pyomo.environ as pyo
import pyomo.opt as po
import random
import json

In [2]:
#READ DATABASE OF COMBOS
Spell_Book=pd.read_csv('Spell_Book _MTG_Commander.csv')

#COMBO IDENTITY
Colors = ['w,g','w', 'g']
Spell_Book=Spell_Book.loc[Spell_Book['Color Identity'].isin(Colors)]

In [3]:
#LIST OF COMBOS
Combo_list=[]

#Take all the combos in the spell book and make a list
for i in range(0,len(Spell_Book)):
    #Go Row by rows, extract columns 1 to 11 (Card Columns)
    Combo=Spell_Book.iloc[[i],1:11]
    #Drop Columns NA
    Combo_2=Combo.dropna(axis='columns')
    #Create a list out the combo with no extra NAN
    Combo_3=list(Combo_2.iloc[0])
    Combo_list.append(Combo_3)

In [4]:
#LIST OF CARDS
list_of_Cards = sum(Combo_list, [])
#RE,PVE A;; TJE DUPLICATES FROM MY DECK
list_of_Cards=list(dict.fromkeys(list_of_Cards))

In [5]:
#CREATE A COMBO VECTOR- MATRIX of 1 and 0. Example CoOMBO 1 is (1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0) , lengh is the lengh of the cards that we are interested ( w, r, w,g)
matrix_list=[]
for i in range(0,len(Combo_list)):
    Combo_Vector=[0]*len(list_of_Cards)
    testing_Combo=Combo_list[i]
    for i in testing_Combo:
        try:
            index=list_of_Cards.index(i)
            Combo_Vector[index]=1
        except:
            pass
    matrix_list.append(Combo_Vector)
#CREATE MATRIX AND MAKE SURE THE NUMBER OF COMBOS - IS IN THE ROWS AND THE CARDS ARE THE COLUMNS   
matrix_form = np.array(matrix_list).transpose()

In [6]:
#ADDING THE ELEMENTS OF THE ROW - IT IS BASICALLY THE NUMBER of CARDS IN EACH COMBO
N_Cards_Combo=matrix_form.transpose().sum(axis=1)

In [7]:
# CREATE A RANDOM VECTOR TO TEST
sample_size=949
labels = [random.randint(0,1) for _ in range(sample_size)]
new_array=np.array([labels])
Gosh=matrix_form*new_array

In [8]:
#THIS IS WHAT WE NEED TO  OPTIMIZE.
result=matrix_form.dot(new_array.transpose())

In [12]:
with open('AllPrices.json') as json_file:
    data = json.load(json_file)
    print(data)




IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Solving the Optimization Problem

#### am going to Optimize is the number of combos in my Deck)


trace(Combo_Matrix * (Vector of Deck) - Sum(Combo_Matrix_Row))









In [24]:
#combos=set(
All_Cards=set(range(1,len(list_of_Cards)))
Number_Combos=set(range(1,matrix_form.shape[0]))
max_cards=100
        

In [ ]:
import pyomo.environ as pyo
import pyomo.opt as po

#Creating asolver
solver=po.SolverFactory('glpk',executable='C:/Users/rvicc/anaconda3/envs/test-ai/Lib/site-packages/pyomo/')

#Initialize enviroment
model = pyo.ConcreteModel()

#Model Range - Number of Cards to bec
model.Number_Combos=py.Set(initialize=Number_Combos)
model.Deck=py.Set(initialize=All_Cards)
model.Deck_size=max_cards

#Initilize the Model
model.c=pyo.Param(model.Number_Combos,model.Deck,initialize=matrix_form,default=0)
model.Deck_size=max_cards

#Pickl the parameters to optimize
model.x=pyo.Var(model.Number_Combos,model.Deck,domain=pyo.Binary)

#Create Variables to Optimize, They are already constrained to 0 and 1
#model.x_1= pyo.Var(within=pyo.Binary)
#model.x_2= pyo.Var(within=pyo.Binary)
#model.x_3= pyo.Var(within=pyo.Binary)
#model.x_4= pyo.Var(within=pyo.Binary)
#model.x_5= pyo.Var(within=pyo.Binary)
#model.x_6= pyo.Var(within=pyo.Binary)
#model.x_7= pyo.Var(within=pyo.Binary)

#obj - combos
expr =sum([sum([model.x_1,model.x_2,model.x_3]),sum([model.x_5,model.x_3])])

#combp
model.con1=pyo.Constraint(expr=33*model.x_1 + 5*model.x_2 +6*model.x_3+ model.x_4 + 3*model.x_5 <=14)
model.con2=pyo.Constraint(expr=model.x_1  + model.x_2 +model.x_3+model.x_4 + model.x_5 <=3)

#Solve it
result=solver.solve(model, tee=True)

In [41]:
import numpy as np
N = 1000000
instruments = ['Fixed Income', 'Equity 1', 'Equity 2', 'Multi-Asset', 'Cash']

# each row is set of weights in order of instruments above
weights = np.zeros(shape=(N, len(instruments)))

weights[:, -1] = 0.05
weights[:, 1] = np.random.uniform(0, 0.3, N)

cols = (0, 2, 3)

# fill columns with random numbers
for col in cols[0:-1]:
    w_remaining = 1 - np.sum(weights, axis=1)
    weights[:, col] = np.random.uniform(0.1, 0.4, N)

# the last column is constrained for normalization
weights[:, cols[-1]] = 1 - np.sum(weights, axis=1)

# select only rows that meet condition:
cond1 = np.all(0.1 <= weights[:, cols], axis=1)
cond2 = np.all(weights[:, cols] <= 0.4, axis=1)
valid_rows = cond1*cond2

weights = weights[valid_rows, :]

# verify sum of weights == 1:
np.testing.assert_allclose(np.sum(weights, axis=1), 1)

 

In [46]:
import numpy as np
N = 1000000
instruments = ['Fixed Income', 'Equity 1', 'Equity 2', 'Multi-Asset', 'Cash']

# each row is set of weights in order of instruments above
weights = np.zeros(shape=(N, len(instruments)))

weights[:, -1] = 0.05
weights[:, 1] = np.random.uniform(0, 0.3, N)

cols = (0, 2, 3)
weights
#fill columns with random numbers
for col in cols[0:-1]:
#    w_remaining = 1 - np.sum(weights, axis=1)
#    weights[:, col] = np.random.uniform(0.1, 0.4, N)
#weights

array([[0.        , 0.23300119, 0.        , 0.        , 0.05      ],
       [0.        , 0.19376315, 0.        , 0.        , 0.05      ],
       [0.        , 0.09335942, 0.        , 0.        , 0.05      ],
       ...,
       [0.        , 0.21700145, 0.        , 0.        , 0.05      ],
       [0.        , 0.12235288, 0.        , 0.        , 0.05      ],
       [0.        , 0.24061703, 0.        , 0.        , 0.05      ]])

In [9]:
N_Simulations=100

list_results=[]
list_Deck=[]
List_Count_Combos=[]
for i in range(0,N_Simulations):
    Random_Deck=np.array(random.sample([0]*(len(list_of_Cards)-100) + [1]*100, len(list_of_Cards)))
    if sum(Random_Deck)==100:
        result_Mult=matrix_form.transpose().dot(Random_Deck)
        #if The Difference is a 0 there is a combo present
        Diference=result_Mult-N_Cards_Combo
        list_Deck.append(Random_Deck)
        List_combo_run=[]
        for j in Diference:
            if j==0:
                List_combo_run.append(j+1)
            else:
                List_combo_run.append(0)
        List_Count_Combos.append(List_combo_run)
        max_result=sum(List_combo_run)   
        list_results.append(max_result)           
    else:
        pass

In [14]:
my_deck=[]
for index,i in enum(list_Deck[2]):
    if i == 1:
        card=list_of_Cards[index]
        my_deck.append(card)
    else:
        pass
my_deck


['Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring Mummy',
 'Sparring M

In [18]:
for i in list_Deck[2]:
    print( i)

0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
1
0
1
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
1
1
0
1
0
0
1
1
0
0
0
0
1
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0


In [33]:
len(list_of_Cards)

583

In [38]:
result_Mult - N_Cards_Combo

ValueError: operands could not be broadcast together with shapes (949,) (583,) 

In [39]:
matrix_form.transpose().shape

(949, 583)

In [40]:
matrix_form.transpose()
Random_Deck.shape

(583,)

In [17]:
list_of_Cards

['Adarkar Valkyrie',
 'Sparring Mummy',
 'Phyrexian Altar',
 'Thornbite Staff',
 'Altar of Dementia',
 "Ashnod's Altar",
 'Saffi Eriksdotter',
 'Karmic Guide',
 'Animation Module',
 'Good-Fortune Unicorn',
 'Antler Skulkin',
 'Solemnity',
 'Genesis Chamber',
 'Argothian Elder',
 'Jungle Basin',
 'Wirewood Lodge',
 'Basalt Monolith',
 'Mana Reflection',
 'Nyxbloom Ancient',
 'Boonweaver Giant',
 'Pattern of Rebirth',
 'Reveillark',
 'Fiend Hunter',
 "Cathars' Crusade",
 'Ornithopter',
 'Crystalline Crawler',
 'Consulate Crackdown',
 'Mycosynth Lattice',
 'Cryptic Trilobite',
 'Archangel of Thune',
 'Sunbond',
 'Staff of Domination',
 'Heliod, Sun-Crowned',
 'Juju Bubble',
 'Devoted Druid',
 'Presence of Gond',
 'Coat of Arms',
 'Vizier of Remedies',
 'Bishop of Wings',
 'Divine Visitation',
 'Drannith Magistrate',
 'Knowledge Pool',
 'Dross Scorpion',
 'Myr Turbine',
 'Fanatical Devotion',
 'Earthcraft',
 'Spawning Grounds',
 'Squirrel Nest',
 'Elvish Aberration',
 'Cloudstone Curio',
 